In [1]:
import numpy as np
import time
import os
import numpy as np
import scipy as sp
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.colors as colors
from random import sample
import csv
from collections import Counter
# Imports from the func.py file
from func import *
import random

In [2]:
# Reads the graph from the graphml file.
G = nx.read_graphml('../data/graph0.graphml', node_type=int)

In [3]:
# Creates an adjacency_list for the graph.
adjacency_list = {}

for node in G.nodes:
    adjacency_list[node] = []

for source, dest in G.edges:
    adjacency_list[source].append(dest)

# Important nodes are computers which are connected to others.
important_nodes = []
for node in adjacency_list:
    if len(adjacency_list[node]) > 0:
        important_nodes.append(node)

In [4]:
# Converts list of tuples into just a list of nodes.
def convertTupleListToList(tuple_list):
    ret = []
    
    for t in tuple_list:
        ret.append(t[1])
        
    return ret

In [5]:
# Generates a ranking of nodes from highest in degree to lowest.
def inDegreeRanking(adjacency_list):
    
    # Counts in degree.
    counter = Counter()
    for node in adjacency_list:
        counter[node] = 0
        for neighbour in adjacency_list[node]:
            counter[neighbour]+=1
    
    # Converts count into list of tuples in format (in degree, node)
    ranking = []
    
    for node in counter:
        ranking.append((counter[node], node))
    
    ranking.sort()
    ranking.reverse()
    
    return convertTupleListToList(ranking)

In [6]:
# Generates a ranking of nodes from highest out degree to lowest.
def outDegreeRanking(adjacency_list):
    
    ranking = []
    for node in adjacency_list:
        ranking.append((len(adjacency_list[node]), node))
    
    ranking.sort()
    ranking.reverse()
        
    return convertTupleListToList(ranking)

In [52]:
# Generates a random ranking of nodes.
def randomRanking(adjacency_list):
    
    ranking = []
    
    for node in adjacency_list:
        ranking.append(node)
    
    random.seed()
    random.shuffle(ranking)
    
    return ranking

In [8]:
def shortestPathBetweennessRanking(G):
    
    perc = .25
    reduced_weight = .1

    shortest_path_betweenness = nx.edge_betweenness_centrality(G)
    A_sp = create_weighted_adjacency_from_edge_betweenness(G, shortest_path_betweenness, perc, weight=reduced_weight)

    v = A_sp.A
    counter = Counter()
    
    ranking = []

    for i in range(len(v)):

        ranking.append((sum(v[i]), i))

    ranking.sort()
    ranking.reverse()

    return convertTupleListToList(ranking)

In [35]:
def localFlowBetweennessRanking(G):

    # Sets up Julia so that we can run Julia files.
    from julia import Julia
    import julia
    # Make sure to change this to your own julia file path.
    julia.install(julia=r"C:\Users\Daniel\AppData\Local\Programs\Julia-1.7.2\bin\julia.exe")
    julia.Julia(runtime=r"C:\Users\Daniel\AppData\Local\Programs\Julia-1.7.2\bin\julia.exe")
    from julia import Main

    # Includes the Julia files.
    Main.include("../julia/local_flow_betweenness.jl");
    
    perc = .25
    reduced_weight = .1

    local50_betweenness = Main.local_flow_betweenness(list(G.nodes()), list(G.edges()), locality_index=.5)
    A_lf50 = create_weighted_adjacency_from_edge_betweenness(G, local50_betweenness, perc, weight=reduced_weight)

    v = A_lf50.A
    counter = Counter()

    ranking = []

    for i in range(len(v)):

        ranking.append((sum(v[i]), i))

    ranking.sort()
    ranking.reverse()
    
    print(ranking)

    return convertTupleListToList(ranking)

In [10]:
def infectNode(adjacency_list, vaccinated, infected):
    
    # List of infectable nodes to select our next infection from.
    infectable = []

    # Go through each infected node and find their neighbours to find nodes to spread to.
    for node in infected:
        for neighbour in adjacency_list[node]:
            if neighbour not in vaccinated and neighbour not in infected:
                infectable.append(neighbour)
        
    # If there are no nodes to infect, return False, indicating that no further nodes can be infected.
    if len(infectable) == 0:
        return False
    
    # Pick a new node randomly from the list of infectable nodes and return True indicating a successful infection.
    infected.add(infectable[random.randrange(0, len(infectable))])
    return True

In [11]:
def vaccinateNode(ranking, vaccinated, infected):
        
    for node in ranking:
        if node not in vaccinated:
            vaccinated.add(node)
            if node in infected:
                infected.remove(node)
            return

In [12]:
def step(ranking, adjacency_list, vaccinated, infected):
    
    done = False
    reward = 0
    
    # If there are no more infections return the reward and say that we're done.
    if not infectNode(adjacency_list, vaccinated, infected):
        done = True
        reward = len(adjacency_list)/num_infected
        
    vaccinateNode(ranking, vaccinated, infected)
    
    return done

In [13]:
# Generates a random ranking of nodes for all the simulations to share.
def generateRandomPolicy(adjacency_list):
    
    ranking = []
    
    for node in adjacency_list:
        ranking.append(node)
    
    random.seed(1)
    random.shuffle(ranking)
    
    return ranking

In [14]:
def finishSimulation(random_policy, node_ranking, adjacency_list, vaccinated, infected, num_infected):
    
    SIMULATION_NUM = 5
    
    # Copy the vaccination so that changes aren't reflected globally.
    vaccinated_copy = vaccinated.copy()
    infected_copy = infected.copy()
    
    # Do a step to test the action we're interested in. If it returns True no new nodes were infected.
    if step(node_ranking, adjacency_list, vaccinated_copy, infected_copy):
        return 0, vaccinated_copy, infected_copy
    
    # Create a copy of the num_infected.
    new_num_infected = num_infected
    done = False
    
    # Repeat the simulation multiple times to account for randomness in spreading.
    for i in range(SIMULATION_NUM):
        
        # Create a copy of the copy of the vaccinated and infected sets for each simulation.
        vaccinated_copy_copy = vaccinated_copy.copy()
        infected_copy_copy = infected_copy.copy()
        
        # While we're still infecting nodes.
        while not done:
            done = step(random_policy, adjacency_list, vaccinated_copy_copy, infected_copy_copy)
            new_num_infected+=1
    
    average_infected = new_num_infected/SIMULATION_NUM
        
    return average_infected, vaccinated_copy, infected_copy

In [15]:
# List of lists where each element is an array paired with a label.
training_data = []
training_label = []

# Number of simulations to build training data from.
simulations = 1

In [60]:
node_ranking = []

node_ranking.append(inDegreeRanking(adjacency_list))
node_ranking.append(outDegreeRanking(adjacency_list))
node_ranking.append(randomRanking(adjacency_list))
node_ranking.append(localFlowBetweennessRanking(G))
node_ranking.append(shortestPathBetweennessRanking(G))

print(node_ranking[1][:42])
print(node_ranking[2][:42])

count = 0

for node in node_ranking[1][:42]:
    if node in node_ranking[2][:42]:
        count+=1
        
print(count)

[(411.90000000000055, 192), (410.1000000000006, 438), (406.5000000000006, 11), (398.6000000000006, 372), (394.80000000000086, 279), (394.8000000000008, 151), (393.9000000000009, 86), (391.2000000000007, 308), (390.9000000000008, 416), (390.3000000000005, 397), (387.6000000000007, 401), (384.00000000000057, 449), (383.1000000000007, 478), (383.1000000000005, 37), (382.3000000000007, 82), (381.30000000000064, 108), (379.5000000000005, 380), (379.5000000000005, 253), (378.6000000000006, 222), (372.30000000000047, 90), (370.7000000000006, 414), (368.6000000000005, 105), (368.1000000000007, 381), (368.00000000000057, 216), (367.7000000000006, 114), (367.10000000000036, 94), (339.50000000000034, 455), (336.00000000000017, 312), (334.3000000000001, 436), (330.6000000000001, 228), (319.70000000000016, 292), (13.0, 142), (10.299999999999999, 239), (10.1, 426), (9.3, 0), (7.4, 177), (5.1, 245), (3.1, 325), (3.0, 56), (3.0, 19), (2.1, 333), (2.1, 181), (1.0, 500), (1.0, 499), (1.0, 498), (1.0, 49

In [45]:
# Training loop
# Maybe try training with every important_node?
for i in range(len(important_nodes)):
    # We have to reset the test so we can move onto next epoch.
#     state, reward, done = env.reset()

    print(f"Got up to node {i}.")

    state = 0
    done = False
    
    infected = set()
    vaccinated = set()
    
    infection_source = important_nodes[i]

#     infection_source = important_nodes[random.randrange(0, len(important_nodes))]
    infected.add(infection_source)
    num_infected = 1
    
    # Array storing the vaccination status of each node.
    vaccination_state = [0]*len(adjacency_list)
    assert len(vaccination_state) == 501
        
    while not done:
        
        # Stores the results of the simulations.
        results = []

        # Create a policy to finish off the simulation for each action.
        random_policy = generateRandomPolicy(adjacency_list)

        # Go through each index which is the "action".
        for action in range(len(node_ranking)):

            # Try randomly finishing the simulation using that action.
            result, vaccinated_copy, infected_copy = finishSimulation(random_policy, node_ranking[action], adjacency_list, vaccinated, infected, num_infected)

            # Add the results to an array to pick from later.
            results.append([result, vaccinated_copy, infected_copy])

        best_result = float('inf')
        best_action = 0
        best_vaccinated = None
        best_infected = None

        # Find the action with best result and save it.
        for i, result in enumerate(results):
            res, vaccinated_copy, infected_copy = result
            if res < best_result:
                best_result = res
                best_action = i
                best_vaccinated = vaccinated_copy
                best_infected = infected_copy
                
        print(best_result, best_action)

        # Update the vaccinated and infected sets from the result we're sticking with.
        vaccinated = best_vaccinated
        infected = best_infected

        # Apply the action to the vaccination_state vector.
        for node in vaccinated:
            vaccination_state[node] = 1
            
        # Append that state and the action to the data.
        training_data.append(vaccination_state)
        training_label.append(best_action)
            
        if num_infected == len(infected):
            done = True
        else:
            num_infected+=1

Got up to node 0.
3.6 2
3.2 2
3.4 0
1.0 0
Got up to node 1.
62.0 0
64.0 0
62.8 1
63.0 0
63.4 0
63.6 1
63.0 2
62.4 2
63.8 2
62.2 0
62.8 2
62.4 2
63.4 2
62.2 2
63.2 0
63.4 2
61.8 0
63.6 2
62.2 2
63.2 0
62.0 1
63.6 1
63.0 0
63.0 0
63.0 0
62.2 2
63.4 0
62.4 1
63.0 0
62.8 2
62.0 0
63.0 1
Got up to node 2.
0.6 0
0.6 0
0 0
Got up to node 3.
63.0 0
62.6 0
62.2 2
63.6 0
63.2 0
63.2 0
62.6 0
62.4 2
62.6 1
63.4 0
63.2 0
63.0 1
62.8 1
62.2 2
61.8 0
61.4 1
62.4 2
63.6 2
62.8 2
62.6 2
63.4 0
62.8 0
63.0 0
63.4 0
63.0 0
62.6 0
63.8 0
63.4 2
62.6 0
63.4 1
62.2 1
Got up to node 4.
0.6 0
0.6 0
0 0
Got up to node 5.
63.2 0
62.6 2
63.0 1
62.0 1
63.6 0
62.2 2
63.4 2
63.4 0
61.6 1
62.6 2
62.6 0
63.0 1
61.8 2
63.6 2
64.2 1
62.4 0
63.0 1
61.8 2
63.2 0
62.0 2
62.6 0
62.2 2
62.6 2
61.8 2
63.2 2
62.8 0
63.2 1
62.2 2
61.6 0
63.2 2
63.0 2
63.0 1
63.2 1
62.8 0
Got up to node 6.
63.0 2
62.8 1
62.2 2
63.0 0
62.0 0
63.0 2
61.8 1
62.6 1
64.0 0
63.0 0
62.2 2
63.6 2
63.0 0
62.4 1
62.2 0
62.6 1
Got up to node 7.
63.2 2
61

In [18]:
import tensorflow as tf
from tensorflow import keras

In [19]:
print(len(training_data))

658


In [20]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(training_data, training_label, test_size = 0.2, random_state = 1)

In [21]:
print(y_train)
print(y_test)

[1, 0, 1, 2, 1, 2, 0, 1, 2, 2, 2, 2, 1, 0, 1, 1, 2, 2, 0, 2, 2, 2, 0, 0, 0, 1, 0, 0, 1, 1, 2, 0, 2, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 2, 1, 0, 0, 2, 0, 2, 0, 0, 0, 1, 2, 2, 0, 1, 0, 0, 2, 1, 0, 2, 2, 2, 2, 0, 2, 1, 1, 1, 2, 0, 1, 0, 1, 1, 1, 1, 2, 0, 1, 1, 0, 0, 2, 1, 0, 2, 0, 2, 0, 2, 2, 2, 2, 0, 2, 1, 2, 2, 2, 1, 1, 0, 0, 1, 1, 1, 1, 2, 0, 2, 1, 1, 2, 0, 2, 0, 1, 2, 0, 1, 2, 1, 1, 0, 0, 0, 2, 2, 0, 1, 2, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 0, 2, 2, 2, 2, 2, 2, 0, 2, 1, 0, 1, 1, 2, 1, 0, 1, 0, 0, 2, 2, 0, 0, 1, 0, 2, 0, 0, 0, 2, 1, 0, 0, 0, 2, 2, 0, 2, 1, 0, 1, 1, 1, 2, 2, 1, 1, 2, 2, 1, 1, 0, 1, 0, 2, 1, 0, 0, 1, 0, 1, 1, 0, 2, 1, 1, 2, 2, 0, 0, 1, 2, 0, 0, 2, 0, 1, 1, 0, 0, 2, 1, 1, 0, 0, 2, 2, 2, 1, 0, 1, 1, 0, 0, 0, 0, 2, 1, 1, 1, 1, 0, 2, 1, 0, 0, 0, 1, 2, 0, 1, 1, 0, 2, 1, 2, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 2, 1, 2, 0, 1, 2, 0, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 2, 0, 0, 1, 2, 0, 1, 2, 2, 0, 1, 2, 1, 1, 0, 1, 0, 1, 2, 1, 0, 0, 2, 0, 0, 1, 2, 1, 2, 0, 0, 1, 1, 0, 1, 0, 1, 1, 

In [22]:
for i in range(len(node_ranking)):
    print(y_train.count(i))

180
193
153


In [23]:
model = keras.Sequential([
    keras.layers.Dense(units=256, activation='relu'),
    keras.layers.Dense(units=128, activation='relu'),
    keras.layers.Dense(units=64, activation='relu'),
    keras.layers.Dense(units=len(node_ranking), activation='softmax')
])

In [24]:
model.compile(optimizer='adam', 
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(
    X_train, y_train, 
    epochs=50, 
    validation_data=(X_test, y_test)
)

Epoch 1/50
17/17 [==============================] - 1s 16ms/step - loss: 1.0874 - accuracy: 0.4240 - val_loss: 1.0846 - val_accuracy: 0.4167
Epoch 2/50
17/17 [==============================] - 0s 2ms/step - loss: 1.0565 - accuracy: 0.4525 - val_loss: 1.0580 - val_accuracy: 0.4470
Epoch 3/50
17/17 [==============================] - 0s 2ms/step - loss: 1.0342 - accuracy: 0.4582 - val_loss: 1.0488 - val_accuracy: 0.4394
Epoch 4/50
17/17 [==============================] - 0s 2ms/step - loss: 1.0141 - accuracy: 0.4696 - val_loss: 1.0448 - val_accuracy: 0.4394
Epoch 5/50
17/17 [==============================] - 0s 2ms/step - loss: 1.0161 - accuracy: 0.4772 - val_loss: 1.0418 - val_accuracy: 0.4242
Epoch 6/50
17/17 [==============================] - 0s 2ms/step - loss: 1.0070 - accuracy: 0.4734 - val_loss: 1.0361 - val_accuracy: 0.4015
Epoch 7/50
17/17 [==============================] - 0s 2ms/step - loss: 1.0025 - accuracy: 0.4696 - val_loss: 1.0488 - val_accuracy: 0.4318
Epoch 8/50
17/17 [=

In [25]:
# Important nodes version.

# Takes the sets of infected and vaccinated nodes and returns the id of a new node to infect.
# Returns None if there is no new node to infect.
def findNewInfection(infected: set, vaccinated: set) -> int:
    
    # List of infectable nodes to select our next infection from.
    infectable = []

    # Go through each infected node and find their neighbours to find nodes to spread to.
    for node in infected:
        for neighbour in adjacency_list[node]:
            if neighbour not in vaccinated and neighbour not in infected:
                infectable.append(neighbour)
        
    # If there are no nodes to infect, return None.
    if len(infectable) == 0:
        return None
    
    # Otherwise, pick a new node randomly from the list of infectable nodes.
    return infectable[random.randrange(0, len(infectable))]

# Takes the list of node rankings, and the number of vaccinated nodes, returns the id of a new node to vaccinate.
def findNewVaccination(node_ranking, vaccinated, vaccination_state) -> int:
    
    prediction = model.predict(np.array([vaccination_state,]))
    vaccination_method = np.argmax(prediction)
    
    index = 0
    while node_ranking[vaccination_method][index] in vaccinated:
        index+=1
        assert index < len(node_ranking[vaccination_method])
        
    return node_ranking[vaccination_method][index]
    
simulation_num = 1

simulations_per_method = simulation_num*len(important_nodes)

# Keep track of the number of infected nodes with each type of vaccination method. 
num_infected_nodes = 0

# Simulate the infection with all nodes as starting nodes.
for count, infection_source in enumerate(important_nodes):
    
    print(f"Started node {count}!")
    
    vaccination_state = np.array([0]*len(adjacency_list))

    # Run simulation_num simulations.
    for i in range(simulation_num):

        # Set random seed to create reproducable results.
        random.seed(i)

        # List containing the current nodes which are infected.
        infected = set()

        # List containing the vaccinated nodes, so we can pick which ones to intervene on.
        vaccinated = set()

        # The source computer starts off as infected.
        infected.add(infection_source)
        num_infected_nodes+=1

        # While the ransomware has spread in the last round, keep going.
        while True:

            new_infection = findNewInfection(infected, vaccinated)

            # If there are no new nodes to infect, end the simulation.
            if new_infection is None:
                break

            # Otherwise, add it to infected.
            infected.add(new_infection)
            num_infected_nodes+=1

            new_vaccination = findNewVaccination(node_ranking, vaccinated, vaccination_state)

            # Vaccinates the selected node.
            vaccinated.add(new_vaccination)
            vaccination_state[new_vaccination] = 1
            if new_vaccination in infected:
                infected.remove(new_vaccination)

print(f"Neural network had an average of {num_infected_nodes/simulations_per_method} infections per simulation.")

Started node 0!
1/1 [==============================] - 0s 9ms/step
Started node 1!
1/1 [==============================] - 0s 9ms/step


1/1 [==============================] - 0s 8ms/step
Started node 2!
1/1 [==============================] - 0s 8ms/step
Started node 3!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
Started node 4!
1/1 [==============================] - 0s 8ms/step
Started node 5!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 9ms/step
Started node 6!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
Started node 7!
1/1 [==============================] - 0s 8ms/step


...

1/1 [==============================] - 0s 8ms/step
Started node 38!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
Started node 39!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
Started node 40!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
Started node 41!
1/1 [==============================] - 0s 8ms/step


1/1 [==============================] - 0s 8ms/step
Neural network had an average of 250.38095238095238 infections per simulation.


In [26]:
print(node_ranking)

[[22, 1, 53, 10, 9, 7, 4, 3, 2, 62, 36, 35, 34, 33, 32, 31, 30, 29, 28, 27, 23, 21, 19, 18, 17, 15, 13, 82, 81, 79, 78, 77, 76, 75, 74, 73, 72, 71, 70, 69, 67, 66, 65, 63, 61, 60, 59, 58, 57, 55, 52, 51, 50, 49, 45, 44, 43, 42, 41, 40, 39, 37, 20, 11, 8, 85, 83, 12, 6, 90, 89, 87, 86, 68, 48, 38, 26, 25, 24, 16, 14, 94, 92, 91, 84, 56, 47, 46, 5, 120, 105, 104, 100, 99, 98, 97, 96, 95, 108, 107, 106, 88, 80, 64, 54, 138, 114, 113, 112, 111, 110, 109, 103, 93, 151, 150, 149, 148, 147, 145, 144, 143, 142, 141, 140, 137, 135, 134, 133, 132, 131, 128, 127, 125, 124, 121, 118, 117, 116, 115, 102, 101, 208, 192, 191, 188, 187, 186, 184, 183, 182, 181, 180, 178, 175, 174, 171, 170, 169, 166, 164, 161, 160, 157, 154, 152, 146, 130, 215, 214, 213, 211, 210, 206, 205, 204, 203, 202, 201, 198, 197, 196, 195, 194, 139, 126, 122, 222, 221, 220, 219, 218, 217, 185, 179, 176, 168, 158, 156, 155, 129, 123, 119, 247, 227, 226, 225, 224, 223, 216, 209, 200, 190, 189, 177, 173, 172, 167, 165, 162, 253, 2

In [27]:
print(f"Neural network had an average of {num_infected_nodes/simulations_per_method} infections per simulation.")

Neural network had an average of 250.38095238095238 infections per simulation.


In [28]:
print(num_infected_nodes)

10516


In [17]:
# Create the graph.
from pyvis.network import Network
net = Network(directed=True)

# Make the vertices
for node in adjacency_list:
    net.add_node(node, label=str(node))

# Make the edges
for node in adjacency_list:
    for neighbour in adjacency_list[node]:
        net.add_edge(node, neighbour, color = 'rgba(255, 99, 71, 0.2)')

In [18]:
# Draw it.
net.toggle_physics(False)
net.show('network.html')

In [43]:
ranking = {}

for node in adjacency_list:
    
    ranking[node] = len(adjacency_list[node])
        
print(ranking)

{1: 11, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 500, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 2, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 500, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 2, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 492, 84: 0, 85: 0, 86: 0, 87: 500, 88: 0, 89: 0, 90: 0, 91: 500, 92: 0, 93: 0, 94: 0, 95: 484, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 490, 107: 0, 108: 0, 109: 500, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 490, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136: 0, 

In [44]:
# Create the graph.
from pyvis.network import Network
net = Network(directed=True)
count = 0

# Make the vertices
for node in adjacency_list:
    if ranking[node] > 1.0:
        count+=1
        net.add_node(node, label=str(node), color = 'rgba(255, 0, 0, 1.0)')
    else:
        net.add_node(node, label=str(node), color = 'rgba(0, 0, 255, 0.2)')

# Make the edges
for node in adjacency_list:
    for neighbour in adjacency_list[node]:
        net.add_edge(node, neighbour, color = 'rgba(0, 0, 0, 0.3)')
        
print(count)

42


In [45]:
# Draw it.
net.toggle_physics(False)
net.show('local_flow_betweenness.html')